In [ ]:
# ============================================================
# Vietnamese Speaker Profiling - Kaggle Training
# ============================================================
!git clone https://github.com/VuThanhLam124/Profiling_gender_dialect.git
!apt-get install -y ffmpeg

Cloning into 'Profiling_gender_dialect'...
remote: Enumerating objects: 153, done.
remote: Counting objects: 100% (153/153), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 153 (delta 89), reused 115 (delta 51), pack-reused 0 (from 0)
Receiving objects: 100% (153/153), 162.15 KiB | 8.11 MiB/s, done.
Resolving deltas: 100% (89/89), done.
remote: Enumerating objects: 153, done.
remote: Counting objects: 100% (153/153), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 153 (delta 89), reused 115 (delta 51), pack-reused 0 (from 0)
Receiving objects: 100% (153/153), 162.15 KiB | 8.11 MiB/s, done.
Resolving deltas: 100% (89/89), done.


In [ ]:
# !rm -rf Profiling_gender_dialect
# !git clone https://github.com/VuThanhLam124/Profiling_gender_dialect.git

Cloning into 'Profiling_gender_dialect'...
remote: Enumerating objects: 98, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (62/62), done.
remote: Enumerating objects: 98, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 98 (delta 51), reused 82 (delta 35), pack-reused 0 (from 0)
Receiving objects: 100% (98/98), 107.13 KiB | 1.47 MiB/s, done.
Resolving deltas: 100% (51/51), done.
remote: Total 98 (delta 51), reused 82 (delta 35), pack-reused 0 (from 0)
Receiving objects: 100% (98/98), 107.13 KiB | 1.47 MiB/s, done.
Resolving deltas: 100% (51/51), done.


In [6]:
cd Profiling_gender_dialect

/kaggle/working/Profiling_gender_dialect


In [7]:
ls

app.py               configs/  finetune.py  notebooks/  requirements.txt
compare_encoders.py  eval.py   infer.py     README.md   src/


In [9]:
# Install dependencies
!pip install -r requirements.txt

In [10]:
!pip install -q transformers==4.44.0 accelerate==0.33.0 datasets==2.21.0
!pip install -q librosa soundfile audiomentations==0.35.0 wandb safetensors

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 101.9 MB/s eta 0:00:0000:0100:01━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/9.5 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 101.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 32.5 MB/s eta 0:00:005l   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/527.3 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 k

In [ ]:
# ============================================================
# TEST 3 ENCODER TYPES (5 epochs each)
# ============================================================
import yaml
import os
import logging

logger = logging.getLogger("speaker_profiling")

ENCODERS_TO_TEST = [
    # "microsoft/wavlm-base-plus",
    # "facebook/hubert-base-ls960",
    "facebook/wav2vec2-base",
    # "facebook/wav2vec2-large-960h"
]

base_config = """
model:
  name: "{encoder_name}"
  num_genders: 2
  num_dialects: 3
  dropout: 0.15
  head_hidden_dim: 256
  freeze_encoder: false 

training:
  batch_size: 32
  gradient_accumulation_steps: 4
  learning_rate: 5e-5
  num_epochs: 5
  warmup_ratio: 0.1
  weight_decay: 0.01
  gradient_clip: 1.0
  lr_scheduler: "linear"
  fp16: true
  dataloader_num_workers: 2

loss:
  dialect_weight: 2.5

wandb:
  enabled: false

data:
  source: "vispeech"
  vispeech_root: "/kaggle/input/vispeech"
  train_meta: "/kaggle/input/vispeech/metadata/trainset.csv"
  train_audio: "/kaggle/input/vispeech/trainset"
  clean_test_meta: "/kaggle/input/vispeech/metadata/clean_testset.csv"
  clean_test_audio: "/kaggle/input/vispeech/clean_testset"
  noisy_test_meta: "/kaggle/input/vispeech/metadata/noisy_testset.csv"
  noisy_test_audio: "/kaggle/input/vispeech/noisy_testset"
  val_split: 0.15

audio:
  sampling_rate: 16000
  max_duration: 5

augmentation:
  enabled: true
  prob: 0.8

output:
  dir: "/kaggle/working/output_{encoder_short}"
  save_total_limit: 1
  metric_for_best_model: "dialect_acc"

early_stopping:
  patience: 3
  threshold: 0.0001

labels:
  gender:
    Male: 0
    Female: 1
  dialect:
    North: 0
    Central: 1
    South: 2

seed: 42
"""

results = {}

for encoder in ENCODERS_TO_TEST:
    encoder_short = encoder.split("/")[-1]
    logger.info("=" * 70)
    logger.info(f"TESTING: {encoder}")
    logger.info("=" * 70)
    
    config_content = base_config.format(
        encoder_name=encoder,
        encoder_short=encoder_short
    )
    
    config_path = f"configs/test_{encoder_short}.yaml"
    with open(config_path, "w") as f:
        f.write(config_content)
    
    logger.info(f"Config saved: {config_path}")
    logger.info(f"freeze_encoder: false (full finetuning)")
    logger.info(f"Training 5 epochs...")
    
    exit_code = os.system(f"python finetune.py --config {config_path}")
    
    if exit_code == 0:
        results[encoder] = "SUCCESS"
        logger.info(f"{encoder_short}: Training completed!")
    else:
        results[encoder] = f"FAILED (exit code: {exit_code})"
        logger.info(f"{encoder_short}: Training failed!")

logger.info("=" * 70)
logger.info("TEST SUMMARY")
logger.info("=" * 70)
for encoder, status in results.items():
    logger.info(f"  {encoder}: {status}")

In [ ]:
# ============================================================
# FINETUNE WITH ViMD (Quick Test - 2 epochs)
# ============================================================
import os

ENCODER = "facebook/wav2vec2-base"
encoder_short = ENCODER.split("/")[-1]

vimd_config = f"""
model:
  name: "{ENCODER}"
  num_genders: 2
  num_dialects: 3
  dropout: 0.15
  head_hidden_dim: 256
  freeze_encoder: false

training:
  batch_size: 16
  gradient_accumulation_steps: 2
  learning_rate: 5e-5
  num_epochs: 2
  warmup_ratio: 0.1
  weight_decay: 0.01
  gradient_clip: 1.0
  lr_scheduler: "linear"
  fp16: true
  dataloader_num_workers: 2

loss:
  dialect_weight: 2.5

wandb:
  enabled: false

data:
  source: "vimd"
  vimd_path: "/kaggle/input/vimd-dataset"

audio:
  sampling_rate: 16000
  max_duration: 5

augmentation:
  enabled: true
  prob: 0.8

output:
  dir: "/kaggle/working/output_vimd"
  save_total_limit: 1
  metric_for_best_model: "dialect_acc"

early_stopping:
  patience: 2
  threshold: 0.001

labels:
  gender:
    Male: 0
    Female: 1
    0: 0
    1: 1
  dialect:
    North: 0
    Central: 1
    South: 2

seed: 42
"""

config_path = "configs/vimd_test.yaml"
with open(config_path, "w") as f:
    f.write(vimd_config)

In [9]:
print(f"Config: {config_path}")
print(f"Encoder: {ENCODER}")
print(f"Epochs: 2 (quick test)")
print("Starting training...")

os.system(f"python finetune.py --config {config_path}")

Config: configs/vimd_test.yaml
Encoder: facebook/wav2vec2-base
Epochs: 2 (quick test)
Starting training...


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]
2025-11-28 08:02:06.906601: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764316926.935429     610 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764316926.944018     610 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
AttributeError: 'MessageFactory' object ha

{'loss': 3.4286, 'grad_norm': 7.337733745574951, 'learning_rate': 2.6595744680851066e-07, 'epoch': 0.0}


  3%|▎         | 50/1878 [01:42<57:00,  1.87s/it]  

{'loss': 3.3493, 'grad_norm': 11.875969886779785, 'learning_rate': 1.196808510638298e-05, 'epoch': 0.05}


...

{'eval_loss': 1.3319623470306396, 'eval_gender_acc': 0.9810526315789474, 'eval_gender_f1': 0.9811645111859427, 'eval_dialect_acc': 0.8031578947368421, 'eval_dialect_f1': 0.7981854470685116, 'eval_combined_f1': 0.8896749791272272, 'eval_runtime': 148.8142, 'eval_samples_per_second': 12.768, 'eval_steps_per_second': 0.8, 'epoch': 2.0}


100%|██████████| 1878/1878 [1:04:41<00:00,  2.07s/it]
2025-11-28 09:07:03 | INFO | Saving model to /kaggle/working/output_vimd/best_model...
INFO:speaker_profiling:Saving model to /kaggle/working/output_vimd/best_model...


{'train_runtime': 3881.6478, 'train_samples_per_second': 7.741, 'train_steps_per_second': 0.484, 'train_loss': 2.1392504211685783, 'epoch': 2.0}


2025-11-28 09:07:04 | INFO | Training completed!
INFO:speaker_profiling:Training completed!


0